In [84]:
%pip install "gymnasium[atari]" ale-py autorom

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\csbro\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
import gymnasium as gym
import ale_py
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, clear_output
import time

gym.register_envs(ale_py)

In [ ]:
def preprocess_frame(frame):
    img = Image.fromarray(frame).convert('L')
    img = img.resize((84, 84), Image.NEAREST)
    return np.array(img, dtype=np.float32) / 255.0

def show_frame_inline(frame_rgb, reward=None, action=None, q_values=None, scale=5, font_size=22):
    # copy frame and resize it
    arr = frame_rgb
    if arr.dtype == np.float32 or arr.dtype == np.float64:
        arr = (np.clip(arr, 0, 1) * 255).astype(np.uint8)
    img = Image.fromarray(arr)
    img = img.resize((img.width*scale, img.height*scale), Image.NEAREST)

    # create context for drawing and font
    draw = ImageDraw.Draw(img)
    font = ImageFont.load_default()

    y = 5
    spacing = font_size + 4

    # draw current reward to gui
    if reward is not None:
        draw.text((5, y), f"Reward: {reward:.2f}", fill=(0,255,0), font=font)
        y += spacing

    # draw current action to gui
    if action is not None:
        draw.text((5, y), f"Action: {action}", fill=(0,255,0), font=font)
        y += spacing

    # draw q values to gui
    if q_values is not None:
        q_str = ", ".join([f"{v:.2f}" for v in q_values])
        draw.text((5, y), f"Top Q: {q_str}", fill=(0,255,0), font=font)

    clear_output(wait=True)
    display(img)


In [ ]:
class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()
        c, h, w = input_shape # channels, height, width
        
        # Convolutional feature extractor
        # ReLU after each layer ensures nonlinearity.
        self.conv = nn.Sequential(
            nn.Conv2d(c, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        # compute conv out size
        with torch.no_grad():
            dummy = torch.zeros(1, c, h, w)
            conv_out_size = int(np.prod(self.conv(dummy).shape[1:]))
        
        # fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def forward(self, x):
        x = self.conv(x)            # pass input through convolutional feature extractor
        x = x.view(x.size(0), -1)   # flatten spacial values into single vector
        return self.fc(x)           # pass flattened values through fully connected layers


In [ ]:
class ReplayBuffer:
    def __init__(self, capacity, state_shape):
        self.capacity = capacity
        self.state_shape = state_shape

        # initialise empties
        self.ptr = 0
        self.size = 0
        self.states = np.zeros((capacity, *state_shape), dtype=np.float32)
        self.next_states = np.zeros((capacity, *state_shape), dtype=np.float32)
        self.actions = np.zeros((capacity,), dtype=np.int64)
        self.rewards = np.zeros((capacity,), dtype=np.float32)
        self.dones = np.zeros((capacity,), dtype=np.float32)

    def push(self, state, action, reward, next_state, done):
        self.states[self.ptr] = state
        self.next_states[self.ptr] = next_state
        self.actions[self.ptr] = action
        self.rewards[self.ptr] = reward
        self.dones[self.ptr] = float(done)
        self.ptr = (self.ptr + 1) % self.capacity
        self.size = min(self.size + 1, self.capacity)

    def sample(self, batch_size):
        # get random batch from buffer
        idx = np.random.choice(self.size, batch_size, replace=False)
        return {
            "states": self.states[idx],
            "actions": self.actions[idx],
            "rewards": self.rewards[idx],
            "next_states": self.next_states[idx],
            "dones": self.dones[idx]
        }

    def __len__(self):
        return self.size

def train_step(q_net, target_net, optimizer, batch, gamma=0.99, device="cpu"):
    # convert all batch lists to tensors
    states = torch.tensor(batch['states'], dtype=torch.float32, device=device)
    actions = torch.tensor(batch['actions'], dtype=torch.long, device=device)
    rewards = torch.tensor(batch['rewards'], dtype=torch.float32, device=device)
    next_states = torch.tensor(batch['next_states'], dtype=torch.float32, device=device)
    dones = torch.tensor(batch['dones'], dtype=torch.float32, device=device)

    # compute q-values for current states
    q_vals = q_net(states)
    q_s_a = q_vals.gather(1, actions.unsqueeze(1)).squeeze(1)

    with torch.no_grad():
        next_q = target_net(next_states)                    # look at the next states
        next_q_max = next_q.max(1)[0]                       # find the best next action
        target = rewards + gamma * next_q_max * (1 - dones) # build the target value

    
    # Huber loss between predicted Q(s,a) and target value
    loss = F.smooth_l1_loss(q_s_a, target)

    # Standard optimizer step
    optimizer.zero_grad()
    loss.backward()

    # Gradient clipping to avoid exploding updates
    torch.nn.utils.clip_grad_norm_(q_net.parameters(), 10.0)

    optimizer.step()
    return loss.item()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Environment
env = gym.make("ALE/Breakout-v5", render_mode="rgb_array")
obs, info = env.reset(seed=42)

# initial lives
lives = info.get('lives', 5)

# state stack init
frame0 = preprocess_frame(obs)
state_stack = np.stack([frame0]*4, axis=0)  # shape (4,84,84)
state = state_stack.copy()

# networks
n_actions = env.action_space.n
q_net = DQN(input_shape=(4,84,84), n_actions=n_actions).to(device)
q_net.train()

target_net = DQN(input_shape=(4,84,84), n_actions=n_actions).to(device)
target_net.load_state_dict(q_net.state_dict())
target_net.eval()

# optimizer
optimizer = optim.Adam(q_net.parameters(), lr=1e-4)

# replay buffer
replay_capacity = 10000
buffer = ReplayBuffer(replay_capacity, state_shape=(4,84,84))

# hyperparams
batch_size = 32                 # size of batches of frames
gamma = 0.80                    # how much the aget cares about future rewards
initial_eps = 1.0               # initial epsilon before decay
final_eps = 0.05                # final epsilon after decay
eps_decay_steps = 1_000_00      # linear decay over this many steps
start_train = 10000             # number of transitions before training starts
train_freq = 4                  # train every n env steps
target_update_freq = 10000      # sync target net every n env steps
max_env_steps = 1_000_000       # total env steps to run
save_path = "dqn_breakout.pth"  # path for saved model

# rewards & punishments
paddle_touch_reward = 0.2
alignment_strength = 0.1
edge_penalty = 0.1
life_lost_penalty = 2.0

# bookkeeping
total_steps = 0
episode = 0
loss_history = []
prev_paddle_center = None
prev_action = None

Using device: cpu


In [ ]:
display_interval = 5
last_display = 0

PADDLE_WIDTH = 16
PADDLE_Y = 193  

while total_steps < max_env_steps:
    # epsilon linear decay
    eps = max(final_eps, initial_eps - (initial_eps - final_eps) * (total_steps / eps_decay_steps))

    # choose action
    if random.random() < eps:
        action = env.action_space.sample()
    else:
        with torch.no_grad():
            st = torch.tensor(state.astype(np.float32), dtype=torch.float32, device=device).unsqueeze(0)
            qvals = q_net(st).cpu().numpy()[0]
            action = int(np.argmax(qvals))

    # step env
    obs, reward, terminated, truncated, info = env.step(action)
    total_steps += 1

    # multiply base block reward
    reward *= 3

    # detect life loss and apply penalty
    new_lives = info.get('lives', lives)
    life_lost = new_lives < lives
    lives = new_lives
    if life_lost:
        reward -= life_lost_penalty

    # Check ball close to paddle vertically
    # access ram values
    ram = env.unwrapped.ale.getRAM()
    ball_x = ram[99]
    ball_y = ram[101]
    paddle_x = ram[72]
    paddle_center = paddle_x + 16/2

    if abs(ball_y - PADDLE_Y) <= 5:
        # Check X overlap with paddle
        if paddle_x - 2 <= ball_x <= paddle_x + PADDLE_WIDTH + 2:
            reward += paddle_touch_reward
    
    # Alignemnt reward
    if prev_paddle_center is None:
        prev_paddle_center = paddle_center
    old_dist = abs(prev_paddle_center - ball_x)
    new_dist = abs(paddle_center - ball_x)
    if new_dist < old_dist:
        reward += alignment_strength
    prev_paddle_center = paddle_center

    # anti edge-hugging
    if paddle_center < 10 or paddle_center > 246:  # screen 0–255
        reward -= edge_penalty

    # preprocess next state
    next_frame = preprocess_frame(obs)
    next_state = np.roll(state, shift=-1, axis=0)
    next_state[-1] = next_frame

    # training 'done' flag
    done_for_training = bool(terminated or truncated)

    # push to replay buffer
    buffer.push(state.copy(), action, reward, next_state.copy(), done_for_training)

    # advance current state
    state = next_state

    # If the actual episode ended then reset environment and bookkeeping
    if terminated or truncated:
        obs, info = env.reset()
        lives = info.get('lives', 5)
        frame0 = preprocess_frame(obs)
        state = np.stack([frame0]*4, axis=0)
        episode += 1

    # Training step
    if len(buffer) >= start_train and (total_steps % train_freq == 0):
        batch = buffer.sample(batch_size)
        loss = train_step(q_net, target_net, optimizer, batch, gamma=gamma, device=device)
        loss_history.append(loss)

    # Update target network periodically
    if total_steps % target_update_freq == 0 and total_steps > 0:
        target_net.load_state_dict(q_net.state_dict())

    # Periodic inline display (less flicker)
    if total_steps - last_display >= display_interval:
        # compute q-values for overlay
        with torch.no_grad():
            st = torch.tensor(state.astype(np.float32), dtype=torch.float32, device=device).unsqueeze(0)
            qvals = q_net(st).cpu().numpy()[0]
        top3 = np.sort(qvals)[-3:][::-1]
        show_frame_inline(obs, reward=reward, action=action, q_values=top3, scale=4, font_size=18)
        last_display = total_steps

    # occasional checkpoint save
    if total_steps % 50000 == 0 and total_steps > 0:
        torch.save({
            "q_net": q_net.state_dict(),
            "target_net": target_net.state_dict(),
            "optimizer": optimizer.state_dict(),
            "total_steps": total_steps
        }, save_path)

# end training loop
env.close()


KeyboardInterrupt: 

In [94]:
gym.register_envs(ale_py)
env = gym.make("ALE/Breakout-v5", render_mode="human")
obs, info = env.reset(seed=0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_shape = (4, 84, 84)
n_actions = env.action_space.n

model = DQN(input_shape, n_actions).to(device)
checkpoint = torch.load("dqn_breakout.pth")
model.load_state_dict(checkpoint["q_net"])
model.eval()

frame = preprocess_frame(obs)
state = np.stack([frame]*4, axis=0)

while True:
    # Convert state to tensor
    s = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

    # Greedy action (no randomness)
    with torch.no_grad():
        q = model(s)
        action = int(torch.argmax(q))

    # Step environment
    obs, reward, terminated, truncated, info = env.step(action)

    # Preprocess new frame
    next_frame = preprocess_frame(obs)
    next_state = np.roll(state, -1, axis=0)
    next_state[-1] = next_frame

    state = next_state

    if terminated or truncated:
        obs, info = env.reset()
        frame = preprocess_frame(obs)
        state = np.stack([frame]*4, axis=0)

: 